In [2]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import tool

In [3]:
import pandas as pd
from qdrant_client import QdrantClient,models
from sentence_transformers import SentenceTransformer
from geopy.distance import geodesic


/home/davis/miniconda3/envs/AI/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
client = QdrantClient(url="http://localhost:6333")
collection_name = "no_coordinates_collection"
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    

/home/davis/miniconda3/envs/AI/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/davis/miniconda3/envs/AI/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
# I USED THIS FOR DEBUGGING
# embeddings = model.encode('900 Embarcadero Del Mar')
# print(len(embeddings))
# output = client.search(
#         collection_name=f"{collection_name}",
#         query_filter=models.Filter(
#             must=[
#                 models.FieldCondition(
#                     key="categories",
#                     match=models.MatchValue(
#                         value='Hotels',
#                     ),
#                 )
#             ]
#         ),
#         query_vector=embeddings,
#         limit=3,
#     )
# print(output)

# Define your tools!

For this we create a Langchain tool for retrieving weather information and use the `convert_to_openai_function` to create the function definition that we will feed into the model

In [6]:
class SearchInput(BaseModel):
    location: str = Field(description="Current location of user.")
    Category: str = Field(description="Type of business that the user wants to go.")
@tool("get_category", args_schema=SearchInput)
def get_category(location: str, Category: str) -> str:
    """Get a business recommendation given location and category of user's choice.\
        e.g. "Name of restaurant": Helena Avenue Bakery, "Location": 131 Anacapa St, Ste C Santa Barbara 93101 ,"Category": food, restaurants, bars """
    # print(location, Category)
    address_embedding = model.encode(location)
    # print(address_embedding)
    output = client.search(
        collection_name=f"{collection_name}",
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="categories",
                    match=models.MatchValue(
                        value=Category,
                    ),
                )
            ]
        ),
        
        query_vector=address_embedding,
        limit=6,
    )

    # print('Query output:',output)
    if len(output) == 0:
        return None
    best_point = max(output, key=lambda x: x.score)
    poi_name = best_point.payload['name']
    poi_address = best_point.payload['full_address']
    latitude = best_point.payload['latitude']
    longitude = best_point.payload['longitude']
    categories = best_point.payload['categories']

    # print(f"The best-rated {Category} is {poi_name} located at {poi_address}.")

    return {"name": poi_name ,"location": poi_address, "latitude": latitude, "longitude": longitude, "categories": categories}

tools = [get_category]
functions = [convert_to_openai_function(t) for t in tools]
functions

[{'name': 'get_category',
  'description': 'get_category(location: str, Category: str) -> str - Get a business recommendation given location and category of user\'s choice.        e.g. "Name of restaurant": Helena Avenue Bakery, "Location": 131 Anacapa St, Ste C Santa Barbara 93101 ,"Category": food, restaurants, bars',
  'parameters': {'type': 'object',
   'properties': {'location': {'description': 'Current location of user.',
     'type': 'string'},
    'Category': {'description': 'Type of business that the user wants to go.',
     'type': 'string'}},
   'required': ['location', 'Category']}}]

# Invoke the Model

Now we can prompt the model and pass the functions 🥳

In [7]:
import ollama
import json

SYSTEM_PROMPT = f"""
You are an navigation assistant with access to these functions -
{json.dumps(functions, indent=4)}
"""

print (SYSTEM_PROMPT)


You are an navigation assistant with access to these functions -
[
    {
        "name": "get_category",
        "description": "get_category(location: str, Category: str) -> str - Get a business recommendation given location and category of user's choice.        e.g. \"Name of restaurant\": Helena Avenue Bakery, \"Location\": 131 Anacapa St, Ste C Santa Barbara 93101 ,\"Category\": food, restaurants, bars",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "description": "Current location of user.",
                    "type": "string"
                },
                "Category": {
                    "description": "Type of business that the user wants to go.",
                    "type": "string"
                }
            },
            "required": [
                "location",
                "Category"
            ]
        }
    }
]



In [8]:
import json
import re
from typing import Optional, Dict

def parse_function_call(input_str: str) -> Optional[Dict[str, any]]:
    """
    Parses a text string to find and extract a function call.
    The function call is expected to be in the format:
    <functioncall> {"name": "<function_name>", "arguments": "<arguments_json_string>"}

    Args:
        input_str (str): The text containing the function call.

    Returns:
        Optional[Dict[str, any]]: A dictionary with 'name' and 'arguments' if a function call is found,
                                  otherwise None.
    """
    # Regex pattern to extract 'name' and 'arguments'
    pattern = r'"name":\s*"([^"]+)",\s*"arguments":\s*\'(.*?)\''

    # Search with regex
    match = re.search(pattern, input_str)
    # print(match)
    if match:
        try:
            name = match.group(1)
            arguments_str = match.group(2)
            arguments = json.loads(arguments_str)
            return {"name": name, "arguments": arguments}
        except json.JSONDecodeError:
            return None
    return None


In [9]:
messages = [
     {'role': 'system','content': SYSTEM_PROMPT}, 
     {'role': 'user','content': 'Im at 900 Embarcadero Del Mar, any cafes nearby?'},
]

response = ollama.chat(model='calebfahlgren/natural-functions', messages=messages)
message = (response['message']['content'])

In [10]:
message

'<functioncall> {"name": "get_category", "arguments": \'{"location": "900 Embarcadero Del Mar", "Category": "cafes"}\'}'

# Call the Function

Here we pull the function out of `kwargs` and call our tool with the arguments

In [11]:
messages.append({'role': 'assistant', 'content': message}) # add ai response to history

function_call = parse_function_call(message) # parse out function call name and args into json
print(function_call)
if function_call and function_call.get("name") == "get_category":
    args = function_call.get("arguments")
    restaurant = get_category.run(args)

restaurant

{'name': 'get_category', 'arguments': {'location': '900 Embarcadero Del Mar', 'Category': 'cafes'}}


{'name': 'The Patio Cafe Santa Barbara',
 'location': '3007 De La Vina St 93105',
 'latitude': 34.4386928,
 'longitude': -119.7285101,
 'categories': ['cafes', 'restaurants', 'breakfast & brunch']}

In [12]:
messages.append({'role': 'user', 'content': 'Function Response: ' + str(restaurant)})
messages

[{'role': 'system',
  'content': '\nYou are an navigation assistant with access to these functions -\n[\n    {\n        "name": "get_category",\n        "description": "get_category(location: str, Category: str) -> str - Get a business recommendation given location and category of user\'s choice.        e.g. \\"Name of restaurant\\": Helena Avenue Bakery, \\"Location\\": 131 Anacapa St, Ste C Santa Barbara 93101 ,\\"Category\\": food, restaurants, bars",\n        "parameters": {\n            "type": "object",\n            "properties": {\n                "location": {\n                    "description": "Current location of user.",\n                    "type": "string"\n                },\n                "Category": {\n                    "description": "Type of business that the user wants to go.",\n                    "type": "string"\n                }\n            },\n            "required": [\n                "location",\n                "Category"\n            ]\n        }\n    

In [13]:
response = ollama.chat(model='calebfahlgren/natural-functions', messages=messages)
response['message']['content']

'The Patio Cafe Santa Barbara is a cafe nearby you at 3007 De La Vina St 93105. You can enjoy their food in the categories of cafes, restaurants, and breakfast & brunch'

# Evalulate the model

In [16]:
def isTruthful(name, address):
    doesPOIExist = client.scroll(
            collection_name=f"{collection_name}",
            scroll_filter  =models.Filter(
                must=[
                    models.FieldCondition(
                        key="name",
                        match=models.MatchValue(
                            value=name,
                        ),
                    ),
                    models.FieldCondition(
                        key="full_address",
                        match=models.MatchValue(
                            value=address,
                        ),
                    )
                ]
            ),
        )

    if doesPOIExist:
        print("Is Truthful: YES")
        return True
    else:
        print("Is Truthful: NO")
        return False

def isNear(location1, location2, minDist):
    distance = geodesic(location1, location2).kilometers
    if (distance > minDist):
        print(f"Is Near: NO, Min Dist is {minDist}km, POI is {distance}km away");
        return False
    print(f"Is Near: YES, Min Dist is {minDist}km, POI is {distance}km away");
    return True

def isSemanticlyRelated(userChosenCategory, outputCategories):
    # print(userChosenCategory)
    # print([cat.lower() for cat in outputCategories])
    if (userChosenCategory in [cat.lower() for cat in outputCategories]):
        print("Is Semantically Related: YES")
        return True
    else:
        print("Is Semantically Related:: NO")
        return False


def generateLLMOutput(message):
    function_call = parse_function_call(message) # parse out function call name and args into json
    print("Input: ", function_call)
    if function_call and function_call.get("name") == "get_category":
        args = function_call.get("arguments")
        output = get_category.run(args)
        if output is None:
            return None
        return output 

    return None


In [18]:
MIN_DIST_KM = 10

def evaluateModel():
    numTruthful = 0
    numNear = 0
    numSemanticallyRelated = 0
    numTested = 0
    locations = [

['49 Glen Annie Rd 93117',  34.435061, -119.872007],
['628 W Micheltorena St 93101',  34.419218, -119.716002],
['38 W Victoria St, Ste 101 93101 ', 34.423608 ,-119.707054],
['7127 Hollister Ave, Ste 30 93117',  34.429918, -119.877502],
 ['163 S Turnpike Rd 93111',  34.437691, -119.790462],
  ['955 La Paz Rd 93108'  ,34.450364 ,-119.661552],
  [ '3739 Mariana Way 93105'  ,34.429008, -119.747950],
    ]

    categories = ['restaurants', 'hotels', 'churches', 'bars', 'food']

    count = 1

    for i in locations:
        for j in categories:
            print(f"Running testcase {count}/{len(locations) * len(categories)} --------------------------------------------------------")
            count += 1

            try:
                response = ollama.chat(model='calebfahlgren/natural-functions', messages=[{'role': 'system','content': SYSTEM_PROMPT},
                                                                                        {'role': 'user','content': f"Im at {i[0]}, any {j} nearby?"}])
                message = (response['message']['content'])
                output = generateLLMOutput(message) 
                if output is None:
                    numTested += 1
                    print('skipping ')
                    continue
                print("Query output:", output)

                if (output and isTruthful(output['name'], output['location'])):
                    numTruthful += 1

                if (output and isNear((i[1], i[2]), (output["latitude"], output['longitude']), MIN_DIST_KM)):
                    numNear += 1

                if (output and isSemanticlyRelated(j, output['categories'])):
                    numSemanticallyRelated += 1
            
            except Exception as e:
                print("Error ", e)
            
            numTested += 1
        
    truthfullnessScore = numTruthful / numTested * 100;
    print("Truthfulness Score: ", truthfullnessScore, "%")

    spatialAwarenessScore = numNear / numTested * 100;
    print("Spatial Awareness Score: ", spatialAwarenessScore, "%")

    semanticRelatedScore = numSemanticallyRelated / numTested * 100;
    print("Semantic Relatedness Score: ", semanticRelatedScore, "%")

evaluateModel()

Running testcase 1/35 --------------------------------------------------------
Input:  {'name': 'get_category', 'arguments': {'location': '49 Glen Annie Rd 93117', 'Category': 'restaurants'}}
Query output: {'name': 'Frog Bar & Grill', 'location': '405 Glen Annie Rd 93117', 'latitude': 34.4429379178, 'longitude': -119.8800369458, 'categories': ['restaurants', 'american (traditional)', 'nightlife', 'bars', 'cocktail bars']}
Is Truthful: YES
Is Near: YES, Min Dist is 10km, POI is 1.1437489122094628km away
Is Semantically Related: YES
Running testcase 2/35 --------------------------------------------------------
Input:  {'name': 'get_category', 'arguments': {'location': '49 Glen Annie Rd 93117', 'Category': 'hotels'}}
Query output: {'name': 'Carpinteria Shores', 'location': '4975 Sandyland Rd 93013', 'latitude': 34.3940407, 'longitude': -119.525464, 'categories': ['hotels', 'hotels & travel', 'event planning & services']}
Is Truthful: YES
Is Near: NO, Min Dist is 10km, POI is 32.1822455172